NEXT STEPS:
- Grasp pose: need to get closer to book? Optimize antipodal grasp? ICP smaller?
- Simulation dies? Check trajectory length?


EXPERIMENTAL CODE QUESTIONS
- Planner: how to design the planner? Is that design the right approach?
- Perception: more than one book? Get ICP for more than one book? How do we segment the point cloud?

## Generating and Visualizing Antipodal Grasps on Book

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import trimesh
import sys
from typing import List 
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RigidTransform,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
    Context,
    Diagram,
    PointCloud,
    Simulator,
    TrajectorySource,
    Solve,
    RotationMatrix,
    MultibodyPlant,
    eq,
    StateInterpolatorWithDiscreteDerivative,
    MinimumDistanceLowerBoundConstraint,
    RollPitchYaw,
    SolutionResult,
    CollisionFilterDeclaration,
    GeometrySet,
    Role,
)
from pydrake.geometry import Meshcat
from pydrake.multibody import inverse_kinematics

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb, AddMultibodyTriad, MakeManipulationStation
from manipulation.utils import ConfigureParser
from manipulation.clutter import GraspCandidateCost, GenerateAntipodalGraspCandidate
from manipulation.icp import IterativeClosestPoint
from manipulation.pick import (
    MakeGripperCommandTrajectory,
    MakeGripperFrames,
    MakeGripperPoseTrajectory,
)
# Own utils
from hwstation.utils import init_diagram, fix_input_port, visualize_diagram, filterCollsionGeometry, get_iiwa_joint_state
from hwstation.add_objects import *
from perception.icp import run_table_book_icp, check_quality_of_icp
from rrt.rrt_planning import *
from grasp.grasping import *

from enum import Enum
import pandas as pd

from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.station import AddPointClouds
from random import random
from manipulation.pick import *
#from manipulation.pick import *

## 1. Setup environment

In [3]:
# Start meshcat
try:
    meshcat = Meshcat(7005)
except:
    pass #This error appears if this cell is executed twice (port 7006 is already taken then)

scenario_data = get_library_scenario_data()
diagram_plan, visualizer, simulator = init_diagram(meshcat, scenario_data)
fix_input_port(diagram_plan, simulator)
diagram_plan_context = diagram_plan.CreateDefaultContext()
scene_graph = diagram_plan.GetSubsystemByName("station").GetSubsystemByName("scene_graph")
sg_context = scene_graph.GetMyContextFromRoot(diagram_plan_context)
filterCollsionGeometry(scene_graph, sg_context)
q0_init = get_iiwa_joint_state(diagram_plan, diagram_plan_context)
simulator.AdvanceTo(1.0)

INFO:drake:Meshcat listening for connections at http://localhost:7005


Station input port size:  <pydrake.systems.framework.InputPort object at 0x7fff8c9d35b0>
plant.GetStateNames():  37
plant.GetActuatorNames():  12
Fixing input port of size:  20
Fixing input port of size:  1


## 2. Running ICP to find point cloud

In [4]:
icp_cloud, X_MS_hat = run_table_book_icp(diagram_plan, diagram_plan_context, meshcat)

fixing improper rotation


## 3. Trajectory Planning with RRT

In [5]:
from manipulation.station import MakeHardwareStation, load_scenario
from manipulation.station import AddPointClouds
from random import random
from manipulation.pick import *
#from manipulation.pick import *

In [6]:
def find_book_grasp_iiwa_config(manipSim: ManipulationStationSim, diagram: Diagram, diagram_context: Context, q0_init: np.ndarray, gripper_setpoint: float):
    found_non_collision_points = False
    while True:
        manipSim.SetStationConfiguration(q0_init, gripper_setpoint)
        optim_grasp = sample_grasps(icp_cloud, diagram, diagram_context)
        q0 = get_iiwa_joint_state(diagram, diagram_context)
        ik_solver = IKSolver()
        q_goal, optimal = ik_solver.solve(optim_grasp, q_guess=q0)
        start_collision = manipSim.ExistsCollision(q0,gripper_setpoint)
        final_collision = manipSim.ExistsCollision(q_goal,gripper_setpoint)
        if (not start_collision) and (not final_collision):
            break
    return q0, q_goal, optim_grasp

In [7]:
# what's different about this function is that now we try to line up with the book but be away from the table
def find_book_grasp_with_y_offset_iiwa_config(manipSim: ManipulationStationSim, diagram: Diagram, diagram_context: Context, q0_init: np.ndarray, gripper_setpoint: float):
    found_non_collision_points = False
    while True:
        manipSim.SetStationConfiguration(q0_init, gripper_setpoint)
        optim_grasp = sample_grasps(icp_cloud, diagram, diagram_context)
        R_optim = optim_grasp.rotation()
        trans_optim = optim_grasp.translation() + np.array([0.0, 0.5, 0.0])
        back_grasp = RigidTransform(R_optim, trans_optim)

        q0 = get_iiwa_joint_state(diagram, diagram_context)
        ik_solver = IKSolver()
        q_goal, optimal = ik_solver.solve(back_grasp, q_guess=q0)
        start_collision = manipSim.ExistsCollision(q0,gripper_setpoint)
        final_collision = manipSim.ExistsCollision(q_goal,gripper_setpoint)
        if (not start_collision) and (not final_collision):
            break
    return q0, q_goal, optim_grasp

In [8]:
#get_joint_limits(diagram_plan.GetSubsystemByName("station").GetSubsystemByName("plant"))

In [9]:
# WE ARE DOING TWO PLANS NOW SO THIS WILL TAKE LONGER TO RUN

gripper_setpoint = 0.1
manipSim = ManipulationStationSim(diagram_plan, diagram_plan_context)

path_found = False 
while not path_found:
    # q0, q_goal, optim_grasp = find_book_grasp_iiwa_config(manipSim, diagram_plan, diagram_plan_context, q0_init,gripper_setpoint)
    q0, q_goal, optim_grasp = find_book_grasp_with_y_offset_iiwa_config(manipSim, diagram_plan, diagram_plan_context, q0_init,gripper_setpoint)
    # the goal here is to line up with the book
    
    iiwa_problem = IiwaProblem(
        q_start=q0,
        q_goal=q_goal,
        gripper_setpoint=gripper_setpoint,
        collision_checker=manipSim,
        debug=True,
    )

    path = rrt_planning(iiwa_problem, 50, 0.05)
    if path is None or len(path) > 20: # reason for the 20 path length is that the path that is a straight shot to the book (not going around cam) will be less than 20
        print("RESTART RRT")
        continue

    # after lining up with the book, try to go into the optimal grasp with a bit of bias of going "into" the book
    R_optim = optim_grasp.rotation()
    trans_optim = optim_grasp.translation() + np.array([-0.0, -0.05, -0.05])
    further_grasp = RigidTransform(R_optim, trans_optim)
    ik_solver = IKSolver()
    q_further, optimal = ik_solver.solve(further_grasp, q_guess=q_goal)
    further_collision = manipSim.ExistsCollision(q_further,gripper_setpoint)
    
    if further_collision:
            print("RESTART RRT due to going into grasp colliding")
            continue
    further_iiwa_problem = IiwaProblem(
        q_start=q_goal,
        q_goal=q_further,
        gripper_setpoint=gripper_setpoint,
        collision_checker=manipSim,
        debug=True,
    )
    further_path = rrt_planning(further_iiwa_problem, 50, 0.05)
    if further_path is None or len(further_path) > 20: # reason for the 20 path length is that the path that is a straight shot to the book (not going around cam) will be less than 20
        print("RESTART RRT due to further path failing")
        continue

    path.extend(further_path)
    
    path_found = True

self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True


In [10]:
# Configure this path and obtain joint and wsg_trajectories
def configure_grasping_path(path_arr, max_time):
    path_arr = np.array(path).T
    wait_at_grasp = len(path_arr[0])
    grasp_pose = path_arr[:, -1]
    grasp_pose = np.tile(grasp_pose, (wait_at_grasp,1)).T
    reversed_path = path_arr[:,::-1]

    # arrive at grasp pose and wait
    complete_path = np.concatenate([path_arr,grasp_pose],axis=1)

    # return from grasp pose
    complete_path = np.concatenate([complete_path,reversed_path],axis=1)

    t_lst = np.linspace(0, max_time, complete_path.shape[1])
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, complete_path)

    pre_pick_time_idx = len(path_arr[0])
    post_pick_time_idx = (int)(pre_pick_time_idx + wait_at_grasp)

    opened = np.array([0.1])
    closed = np.array([0.0])

    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
        [t_lst[0], t_lst[pre_pick_time_idx]],
        np.hstack([[opened], [opened]]),
    )
    traj_wsg_command.AppendFirstOrderSegment(t_lst[post_pick_time_idx], closed)
    traj_wsg_command.AppendFirstOrderSegment(t_lst[-1], closed)

    return q_traj, traj_wsg_command

In [11]:
max_time = 10.0
q_traj, traj_wsg_command = configure_grasping_path(path, max_time)

## 4. Executing trajectory

In [12]:
scenario_data = get_library_scenario_data()
meshcat.Delete()
builder = DiagramBuilder()
scenario = load_scenario(data=scenario_data)
station = builder.AddSystem(MakeHardwareStation(scenario, meshcat,parser_preload_callback=ConfigureParser))


# COMMENTING OUT THE POINT CLOUD STUFF FOR NOW FOR THE PURPOSE OF VISUALIZING
# # Adding point cloud extractors:
# to_point_cloud = AddPointClouds(
#     scenario=scenario, station=station, builder=builder, meshcat=meshcat
# )

# # Connect point clouds with output port:
# for idx, name in enumerate(to_point_cloud.keys()):
#     builder.ExportOutput(
#         to_point_cloud[name].get_output_port(), name+"_ptcloud")

builder.ExportOutput(
    station.GetOutputPort("mobile_iiwa.state_estimated"),
    "mobile_iiwa.state_estimated"
)

# builder.ExportInput(
#     station.GetInputPort("mobile_iiwa.desired_state"),
#     "mobile_iiwa.desired_state"
# )

q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
g_traj_system = builder.AddSystem(TrajectorySource(traj_wsg_command))

state_interpolator = builder.AddSystem(StateInterpolatorWithDiscreteDerivative(num_positions=10, time_step=0.1))

# builder.Connect(
#     q_traj_system.get_output_port(), station.GetInputPort("mobile_iiwa.desired_state")
# )

builder.Connect(
    q_traj_system.get_output_port(), state_interpolator.GetInputPort("position")
)

builder.Connect(
    state_interpolator.GetOutputPort("state"), station.GetInputPort("mobile_iiwa.desired_state")
)

builder.Connect(
    g_traj_system.get_output_port(), station.GetInputPort("wsg.position")
)


visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)
diagram = builder.Build()
#diagram.set_name("plant and scene_graph")
diagram_context = diagram.CreateDefaultContext()
scene_graph = diagram.GetSubsystemByName("station").GetSubsystemByName("scene_graph")
sg_context = scene_graph.GetMyContextFromRoot(diagram_context)
filterCollsionGeometry(scene_graph, sg_context)

simulator = Simulator(diagram)
#fix_input_port(diagram, simulator)
visualizer.StartRecording(False)
simulator.AdvanceTo(10.0)
    

In [13]:
# Run this to see what happened
visualizer.PublishRecording()

In [14]:
from grasp.grasping import *
from hwstation.utils import visualize_frame

In [54]:
placement_frame = get_shelf_placement_frame(diagram, diagram_context, "B", row=1, column=1)
world_to_gripper = RigidTransform(RotationMatrix(np.array([[0,1,0],[1,0,0],[0,0,-1]])))
placement_frame = RigidTransform(placement_frame.rotation(),
p=placement_frame.translation() + np.array([-SHELF_WIDTH/5,0,-0.00])) #np.array([-SHELF_WIDTH/5,0,-0.05])) 
placement_frame = placement_frame.multiply(world_to_gripper)
visualize_frame("placement_frame", meshcat, placement_frame)
preplacement_frame = compute_pre_placement_frame(placement_frame) 
visualize_frame("preplacement_frame", meshcat, preplacement_frame)

In [55]:
def clip_to_joint_limits(q_config, joint_limits, tol=0.01):
    q_config = q_config.copy()
    upper_mask = (q_config>joint_limits[:,1])
    lower_mask = (q_config<joint_limits[:,0])
    upper_distance_mask = np.abs(q_config-joint_limits[:,1])
    lower_distance_mask = np.abs(q_config-joint_limits[:,0])
    upper_mask = (upper_mask & (upper_distance_mask<tol))
    lower_mask = (lower_mask & (lower_distance_mask<tol))
    q_config[upper_mask] = joint_limits[upper_mask,1]
    q_config[lower_mask] = joint_limits[lower_mask,0]
    return q_config

In [56]:
# what's different about this function is that now we try to line up with the book but be away from the table
def find_shelf_drop_grasp(manipSim: ManipulationStationSim, diagram: Diagram, diagram_context: Context, q0_init: np.ndarray, gripper_setpoint: float ,placement_frame: RigidTransform, preplacement_frame: RigidTransform):
    joint_limits = get_joint_limits(diagram.GetSubsystemByName("station").GetSubsystemByName("plant"))
    while True:
        manipSim.SetStationConfiguration(q0_init, gripper_setpoint)
        q0 = get_iiwa_joint_state(diagram, diagram_context)
        ik_solver = IKSolver()
        q_middle_goal, optimal = ik_solver.solve(preplacement_frame, q_guess=q0)
        #Clip to join limits (may be cause by numerical errors)
        q0 = clip_to_joint_limits(q0, joint_limits)
        q_middle_goal = clip_to_joint_limits(q_middle_goal, joint_limits)

        start_collision = manipSim.ExistsCollision(q0,gripper_setpoint)
        middle_collision = manipSim.ExistsCollision(q_middle_goal,gripper_setpoint)
        
        if (not start_collision) and (not middle_collision):
            manipSim.SetStationConfiguration(q_middle_goal, gripper_setpoint)
            q_middle = get_iiwa_joint_state(diagram, diagram_context)
            ik_solver = IKSolver()
            q_guess = q_middle.copy()
            #q_guess[0] = q_guess[0] - SHELF_WIDTH/2
            q_goal, optimal = ik_solver.solve(placement_frame, q_guess=q_guess)
            #Clip to join limits (may be cause by numerical errors)
            q_goal = clip_to_joint_limits(q_goal, joint_limits)
            q_middle = clip_to_joint_limits(q_middle, joint_limits)

            middle_collision = manipSim.ExistsCollision(q_middle,gripper_setpoint)
            final_collision = manipSim.ExistsCollision(q_goal,gripper_setpoint)
            if (not middle_collision) and (not final_collision):
                break
    return q0, q_middle_goal, q_goal

In [60]:
q0_post_grasping = get_iiwa_joint_state(diagram, diagram_context)
q0_post_grasping, q_middle_goal, q_goal = find_shelf_drop_grasp(manipSim, diagram_plan, diagram_plan_context,q0_post_grasping,0.1,placement_frame, preplacement_frame)

In [61]:
# q_middle_goal[4:] = q0_post_grasping[4:].copy()
# joint_limits_dict = dict(zip([f"iiwa_joint_{idx}" for idx in range(2,8)],
# [[q0_index-0.01,q0_index+0.01] for q0_index in q0_post_grasping[4:]]
# ))
# joint_limits_test = get_joint_limits(diagram.GetSubsystemByName("station").GetSubsystemByName("plant"),
# joint_limits_dict=joint_limits_dict)
joint_limits_dict = {}

In [62]:
# WE ARE DOING TWO PLANS NOW SO THIS WILL TAKE LONGER TO RUN
path_found = False 
while not path_found:
    iiwa_problem = IiwaProblem(
        q_start=q0_post_grasping,
        q_goal=q_middle_goal,
        gripper_setpoint=gripper_setpoint,
        collision_checker=manipSim,
        debug=True,
        joint_limits_dict=joint_limits_dict,
    )

    path = rrt_planning(iiwa_problem, 1000, 0.5)
    if path is None: # reason for the 20 path length is that the path that is a straight shot to the book (not going around cam) will be less than 20
        print("RESTART RRT")
        continue
    print("Finished first path")
    further_iiwa_problem = IiwaProblem(
        q_start=q_middle_goal,
        q_goal=q_goal,
        gripper_setpoint=gripper_setpoint,
        collision_checker=manipSim,
        debug=True,
    )
    further_path = rrt_planning(further_iiwa_problem, 200, 0.20)
    if further_path is None: # reason for the 20 path length is that the path that is a straight shot to the book (not going around cam) will be less than 20
        print("RESTART RRT due to further path failing")
        continue

    path.extend(further_path)
    
    path_found = True

self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True
Finished first path
self.collide:  False
self.valid configuration:  True
self.collide:  False
self.valid configuration:  True


In [63]:
# Configure this path and obtain joint and wsg_trajectories
def configure_grasping_path(path_arr, max_time):
    path_arr = np.array(path).T
    wait_at_grasp = len(path_arr[0])
    grasp_pose = path_arr[:, -1]
    grasp_pose = np.tile(grasp_pose, (wait_at_grasp,1)).T
    reversed_path = path_arr[:,::-1]

    # arrive at grasp pose and wait
    complete_path = np.concatenate([path_arr,grasp_pose],axis=1)

    # return from grasp pose
    complete_path = np.concatenate([complete_path,reversed_path],axis=1)

    t_lst = np.linspace(0, max_time, complete_path.shape[1])
    q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, complete_path)

    pre_pick_time_idx = len(path_arr[0])
    post_pick_time_idx = (int)(pre_pick_time_idx + wait_at_grasp)

    opened = np.array([0.1])
    closed = np.array([0.0])

    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(
        [t_lst[0], t_lst[pre_pick_time_idx]],
        np.hstack([[closed], [closed]]),
    )
    traj_wsg_command.AppendFirstOrderSegment(t_lst[post_pick_time_idx], opened)
    traj_wsg_command.AppendFirstOrderSegment(t_lst[-1], opened)

    return q_traj, traj_wsg_command

In [64]:
q_traj, traj_wsg_command = configure_grasping_path(path,max_time=10.0)

In [65]:
scenario_data = get_library_scenario_data()
meshcat.Delete()
builder = DiagramBuilder()
scenario = load_scenario(data=scenario_data)
station = builder.AddSystem(MakeHardwareStation(scenario, meshcat,parser_preload_callback=ConfigureParser))


# COMMENTING OUT THE POINT CLOUD STUFF FOR NOW FOR THE PURPOSE OF VISUALIZING
# # Adding point cloud extractors:
# to_point_cloud = AddPointClouds(
#     scenario=scenario, station=station, builder=builder, meshcat=meshcat
# )

# # Connect point clouds with output port:
# for idx, name in enumerate(to_point_cloud.keys()):
#     builder.ExportOutput(
#         to_point_cloud[name].get_output_port(), name+"_ptcloud")

builder.ExportOutput(
    station.GetOutputPort("mobile_iiwa.state_estimated"),
    "mobile_iiwa.state_estimated"
)

# builder.ExportInput(
#     station.GetInputPort("mobile_iiwa.desired_state"),
#     "mobile_iiwa.desired_state"
# )

q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
g_traj_system = builder.AddSystem(TrajectorySource(traj_wsg_command))

state_interpolator = builder.AddSystem(StateInterpolatorWithDiscreteDerivative(num_positions=10, time_step=0.1))

# builder.Connect(
#     q_traj_system.get_output_port(), station.GetInputPort("mobile_iiwa.desired_state")
# )

builder.Connect(
    q_traj_system.get_output_port(), state_interpolator.GetInputPort("position")
)

builder.Connect(
    state_interpolator.GetOutputPort("state"), station.GetInputPort("mobile_iiwa.desired_state")
)

builder.Connect(
    g_traj_system.get_output_port(), station.GetInputPort("wsg.position")
)


visualizer = MeshcatVisualizer.AddToBuilder(builder, station.GetOutputPort("query_object"), meshcat)
diagram = builder.Build()
#diagram.set_name("plant and scene_graph")
diagram_context = diagram.CreateDefaultContext()
scene_graph = diagram.GetSubsystemByName("station").GetSubsystemByName("scene_graph")
sg_context = scene_graph.GetMyContextFromRoot(diagram_context)
filterCollsionGeometry(scene_graph, sg_context)

simulator = Simulator(diagram)
#fix_input_port(diagram, simulator)
visualizer.StartRecording(False)
simulator.AdvanceTo(10.0)

In [66]:
visualizer.PublishRecording()

## CODE BELOW THIS POINT IS EXPERIMENTAL

In [24]:
class PlannerState(Enum):
    WAIT_FOR_OBJECTS_TO_SETTLE = 1
    GRASPING_BOOK_ON_TABLE = 2
    GOING_TO_SHELF = 3
    PLACING_BOOK_ON_SHELF = 4
    GO_BACK_TO_TABLE = 5

In [25]:
class Planner(LeafSystem):
    
    """A Planner system.

    Input Ports:
    - body_poses: RigidTransforms given the desired frame of the gripper
    - table_book_grasp: (cost,RigidTransform) giving the desired frame to grasp a book on the table
    - shelf_book_grasp: (cost, RididTransform) giving the desired frame to put a book on the shelf
    - wsg_state: a vector of size 2 giving the WSG commands (I supose each fingers position)
    - iiwa_joints: a vector of size 7 for joint positions
    - iiwa_base: a vector of size 2 for x- and y- coordinates of base

    States:
    - self._mode_index: current mode of the planner
    - self._traj_X_G_index: trajectory of gripper frame
    - self._traj_wsg_index: command trajectory of WSG fingers?
    - self._times_index: dictionary of times
    - self._attempts_index: attempts of grasping something

    Output ports:
    - 
    -
    -
    """

    def __init__(self, plant):
        LeafSystem.__init__(self)

        self._gripper_body_index = plant.GetBodyByName("body").index()
        
        #********
        #Declare input ports
        #********
        self.DeclareAbstractInputPort(
            "body_poses", AbstractValue.Make([RigidTransform()])
        )
        self._table_book_grasp_index = self.DeclareAbstractInputPort(
            "table_book_grasp", AbstractValue.Make((np.inf, RigidTransform()))
        ).get_index()

        self._shelf_book_grasp_index = self.DeclareAbstractInputPort(
            "shelf_book_grasp", AbstractValue.Make((np.inf, RigidTransform()))
        ).get_index()

        #Declares a vector-valued input port of size 2:
        self._wsg_state_index = self.DeclareVectorInputPort(
            "wsg_state", 2
        ).get_index()

        # For GoHome mode.
        num_iiwa_actuators = 10
        self._iiwa_actuators_index = self.DeclareVectorInputPort(
            "iiwa_actuators", num_iiwa_actuators
        ).get_index()

        #********
        #Declare states of the system
        #********

        #Abstract state giving the mode:
        #A Context obtained with CreateDefaultContext() will contain this abstract state 
        #variable initially set to a clone of the model_value given here. The actual concrete 
        # type is always preserved.
        self._mode_index = self.DeclareAbstractState(
            AbstractValue.Make(PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE)
        )
        self._q_traj_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial())
        )
        self._traj_wsg_index = self.DeclareAbstractState(
            AbstractValue.Make(PiecewisePolynomial())
        )
        self._times_index = self.DeclareAbstractState(
            AbstractValue.Make({"initial": 0.0})
        )
        #Declares a discrete state group with num_state_variables state variables, 
        #stored in a BasicVector initialized to be all-zero.
        self._attempts_index = self.DeclareDiscreteState(1)

        #********
        #Declare output ports of the system
        #********
        # self.DeclareAbstractOutputPort(
        #     "X_WG",
        #     lambda: AbstractValue.Make(RigidTransform()),
        #     self.CalcGripperPose)

        # self.DeclareVectorOutputPort("wsg_position", 1, self.CalcWsgPosition)

        # self.DeclareAbstractOutputPort(
        #     "control_mode",
        #     lambda: AbstractValue.Make(InputPortIndex(0)),
        #     self.CalcControlMode,
        # )

        # self.DeclareAbstractOutputPort(
        #     "reset_diff_ik",
        #     lambda: AbstractValue.Make(False),
        #     self.CalcDiffIKReset,
        # )

        # self._q0_index = self.DeclareDiscreteState(num_positions)  # for q0
        # self._traj_q_index = self.DeclareAbstractState(
        #     AbstractValue.Make(PiecewisePolynomial())
        # )

        self.DeclareVectorOutputPort(
            "iiwa_command", 10, self.CalcIiwa
        )

        #********
        #Declare initialization
        #********
        self.DeclareInitializationDiscreteUpdateEvent(self.Initialize)

        #********
        #Declare periodic updates
        #********
        # Declares that an UnrestrictedUpdate event should occur periodically and 
        # that it should invoke the Update method every 0.1 second at 0.0 offset
        self.DeclarePeriodicUnrestrictedUpdateEvent(0.1, 0.0, self.Update)

    def Update(self, context, state):
        """Function to updates the mode what we are doing. We do not update the output here 
        - just the state (calling plan if something goes wrong)"""

        #Derive what mode we are in:
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        # Get time:
        current_time = context.get_time()

        # Get all "milestone" times:
        times = context.get_abstract_state(int(self._times_index)).get_value()

        #Wait for objects to settle:
        if mode == PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE:
            # Are we more than 1s past the initial time? If yes, move on:
            if context.get_time() - times["initial"] > 1.0:
                self.Plan(context, state)
            return
        
        elif mode == PlannerState.GO_HOME:
            sys.exit("GOING HOME IS NOT IMPLEMENTED YET.")
            # traj_q = context.get_mutable_abstract_state(
            #     int(self._traj_q_index)
            # ).get_value()
            # if not traj_q.is_time_in_range(current_time):
            #     self.Plan(context, state)
            return

        # If we are between pick and place and the gripper is closed, then
        # we've missed or dropped the object.  Time to replan.
        if (
            current_time > times["postpick"]
            and current_time < times["preplace"]
        ):
            pass
            # Get teh state of the WSG
            # wsg_state = self.get_input_port(self._wsg_state_index).Eval(
            #     context
            # )
            # Check if WSG is closed all the way (no objects between fingers)
            # if wsg_state[0] < 0.01:
            #     attempts = state.get_mutable_discrete_state(
            #         int(self._attempts_index)
            #     ).get_mutable_value()
            #     if attempts[0] > 10:
            #         sys.exit("More than 10 attempts.")
            #     attempts[0] += 1

            #     # Reset to starting point to start anew
            #     state.get_mutable_abstract_state(
            #         int(self._mode_index)
            #     ).set_value(PlannerState.WAIT_FOR_OBJECTS_TO_SETTLE)
            #     times = {"initial": current_time}
            #     state.get_mutable_abstract_state(
            #         int(self._times_index)
            #     ).set_value(times)

            #     # Stop moving and stay where u are
            #     #DEBUG HERE:
            #     X_G = self.get_input_port(0).Eval(context)[
            #         int(self._gripper_body_index)
            #     ]
            #     state.get_mutable_abstract_state(
            #         int(self._q_traj_index)
            #     ).set_value(
            #         PiecewisePolynomial.MakeLinear(
            #             [current_time, np.inf], [X_G, X_G]
            #         )
            #     )
            #     return

        traj_q = context.get_abstract_state(
            int(self._q_traj_index)
        ).get_value()
        if not traj_q.is_time_in_range(current_time):
            self.Plan(context, state)
            return

        q_curr = self.get_input_port(0).Eval(context)[
            int(self._iiwa_actuators_index)
        ]

        if (
            np.linalg.norm(
                traj_q.GetPose(current_time)
                - q_curr
            )
            > 0.5
        ):
            # If my trajectory tracking has gone this wrong, then I'd better
            # stop and replan.  TODO(russt): Go home, in joint coordinates,
            # instead.
            print("WARNING: Trajectory tracking has gone wrong.")
            self.GoHome(context, state)
            return

    def GoHome(self, context, state):
        print("Replanning due to large tracking error.")
        
        # Set mode to GO_TO_HOME_POSITION
        state.get_mutable_abstract_state(int(self._mode_index)).set_value(
            PlannerState.GO_TO_HOME_POSITION
        )
        
        # Get current joint position
        q = self.get_input_port(self._iiwa_position_index).Eval(context)
        
        # Get the initial position
        q0 = copy(context.get_discrete_state(self._q0_index).get_value())
        q0[0] = q[0] # Don't change x
        q0[1] = q[1] # Don't change y
        q0[2] = q[2] # Don't change z
        q0[3] = q[3]  # Safer to not reset the first joint. Don't swing the arm around

        current_time = context.get_time()
        q_traj = PiecewisePolynomial.FirstOrderHold(
            [current_time, current_time + 5.0], np.vstack((q, q0)).T
        )
        state.get_mutable_abstract_state(int(self._traj_q_index)).set_value(
            q_traj
        )

    def Plan(self, context, state):
        mode = copy(
            state.get_mutable_abstract_state(int(self._mode_index)).get_value()
        )

        X_G = {
            "initial": self.get_input_port(0).Eval(context)[
                int(self._gripper_body_index)
            ]
        }

        cost = np.inf
        for i in range(5):
            if mode == PlannerState.PICKING_FROM_Y_BIN:
                cost, X_G["pick"] = self.get_input_port(
                    self._y_bin_grasp_index
                ).Eval(context)
                if np.isinf(cost):
                    mode = PlannerState.PICKING_FROM_X_BIN
            else:
                cost, X_G["pick"] = self.get_input_port(
                    self._x_bin_grasp_index
                ).Eval(context)
                if np.isinf(cost):
                    mode = PlannerState.PICKING_FROM_Y_BIN
                else:
                    mode = PlannerState.PICKING_FROM_X_BIN

            if not np.isinf(cost):
                break

        assert not np.isinf(
            cost
        ), "Could not find a valid grasp in either bin after 5 attempts"
        state.get_mutable_abstract_state(int(self._mode_index)).set_value(mode)

        # TODO(russt): The randomness should come in through a random input
        # port.
        if mode == PlannerState.PICKING_FROM_X_BIN:
            # Place in Y bin:
            X_G["place"] = RigidTransform(
                RollPitchYaw(-np.pi / 2, 0, 0),
                [rng.uniform(-0.25, 0.15), rng.uniform(-0.6, -0.4), 0.3],
            )
        else:
            # Place in X bin:
            X_G["place"] = RigidTransform(
                RollPitchYaw(-np.pi / 2, 0, np.pi / 2),
                [rng.uniform(0.35, 0.65), rng.uniform(-0.12, 0.28), 0.3],
            )

        X_G, times = MakeGripperFrames(X_G, t0=context.get_time())
        print(
            f"Planned {times['postplace'] - times['initial']} second trajectory in mode {mode} at time {context.get_time()}."
        )
        state.get_mutable_abstract_state(int(self._times_index)).set_value(
            times
        )

        if False:  # Useful for debugging
            AddMeshcatTriad(meshcat, "X_Oinitial", X_PT=X_O["initial"])
            AddMeshcatTriad(meshcat, "X_Gprepick", X_PT=X_G["prepick"])
            AddMeshcatTriad(meshcat, "X_Gpick", X_PT=X_G["pick"])
            AddMeshcatTriad(meshcat, "X_Gplace", X_PT=X_G["place"])

        traj_X_G = MakeGripperPoseTrajectory(X_G, times)
        traj_wsg_command = MakeGripperCommandTrajectory(times)

        state.get_mutable_abstract_state(int(self._traj_X_G_index)).set_value(
            traj_X_G
        )
        state.get_mutable_abstract_state(int(self._traj_wsg_index)).set_value(
            traj_wsg_command
        )

    def start_time(self, context):
        return (
            context.get_abstract_state(int(self._q_traj_index))
            .get_value()
            .start_time()
        )

    def end_time(self, context):
        return (
            context.get_abstract_state(int(self._q_traj_index))
            .get_value()
            .end_time()
        )

    def CalcGripperPose(self, context, output):
        context.get_abstract_state(int(self._mode_index)).get_value()

        traj_X_G = context.get_abstract_state(
            int(self._traj_X_G_index)
        ).get_value()
        if traj_X_G.get_number_of_segments() > 0 and traj_X_G.is_time_in_range(
            context.get_time()
        ):
            # Evaluate the trajectory at the current time, and write it to the
            # output port.
            output.set_value(
                context.get_abstract_state(int(self._traj_X_G_index))
                .get_value()
                .GetPose(context.get_time())
            )
            return

        # Command the current position (note: this is not particularly good if the velocity is non-zero)
        output.set_value(
            self.get_input_port(0).Eval(context)[int(self._gripper_body_index)]
        )

    def CalcWsgPosition(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()
        opened = np.array([0.107])
        np.array([0.0])

        if mode == PlannerState.GO_HOME:
            # Command the open position
            output.SetFromVector([opened])
            return

        traj_wsg = context.get_abstract_state(
            int(self._traj_wsg_index)
        ).get_value()
        if traj_wsg.get_number_of_segments() > 0 and traj_wsg.is_time_in_range(
            context.get_time()
        ):
            # Evaluate the trajectory at the current time, and write it to the
            # output port.
            output.SetFromVector(traj_wsg.value(context.get_time()))
            return

        # Command the open position
        output.SetFromVector([opened])

    def CalcControlMode(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        if mode == PlannerState.GO_HOME:
            output.set_value(InputPortIndex(2))  # Go Home
        else:
            output.set_value(InputPortIndex(1))  # Diff IK

    def CalcDiffIKReset(self, context, output):
        mode = context.get_abstract_state(int(self._mode_index)).get_value()

        if mode == PlannerState.GO_HOME:
            output.set_value(True)
        else:
            output.set_value(False)

    def Initialize(self, context, discrete_state):
        discrete_state.set_value(
            int(self._q0_index),
            self.get_input_port(int(self._iiwa_position_index)).Eval(context),
        )

    def CalcIiwaBase(self, context, output):
        pass #Not implemented yet
        # traj_q = context.get_mutable_abstract_state(
        #     int(self._traj_q_index)
        # ).get_value()

        # output.SetFromVector(traj_q.value(context.get_time()))

    def CalcIiwaJoint(self, context, output):
        traj_q = context.get_mutable_abstract_state(
            int(self._traj_q_index)
        ).get_value()

        output.SetFromVector(traj_q.value(context.get_time()))



NameError: name 'LeafSystem' is not defined

In [ ]:
# def BuildAndSimulateTrajectory(q_traj, g_traj, duration=0.01):
#     """Simulate trajectory for manipulation station.
#     @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
#     @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
#     """
#     builder = DiagramBuilder()
#     station = builder.AddSystem(
#         MakeManipulationStation(
#             filename="package://manipulation/manipulation_station_with_cupboard.dmd.yaml",
#             time_step=1e-3,
#         )
#     )
#     plant = station.GetSubsystemByName("plant")
#     scene_graph = station.GetSubsystemByName("scene_graph")
#     AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

duration = 1.0
q_traj_system = builder.AddSystem(TrajectorySource(trajectory))
g_traj_system = builder.AddSystem(TrajectorySource(traj_wsg_command))

visualizer = MeshcatVisualizer.AddToBuilder(
    environment_builder,
    environment_station.GetOutputPort("query_object"),
    meshcat,
    MeshcatVisualizerParams(delete_on_initialization_event=False),
)

environment_builder.Connect(
    q_traj_system.get_output_port(), environment_station.GetInputPort("iiwa_position")
)
environment_builder.Connect(
    g_traj_system.get_output_port(), environment_station.GetInputPort("wsg_position")
)

#diagram = environment_builder.Build()

simulator = Simulator(environment)
visualizer.StartRecording(False)
simulator.AdvanceTo(duration)
visualizer.PublishRecording()

SystemExit: Failure at systems/primitives/trajectory_source.cc:25 in TrajectorySource(): condition 'trajectory.cols() == 1' failed.

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from pydrake.multibody import inverse_kinematics

In [ ]:
max_time = max(traj_times.values())
t_lst = np.linspace(0, max_time, 30)
pose_lst = []
for t in t_lst:
    #AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePose(t), opacity=0.2)
    pose_lst.append(RigidTransform(trajectory.value(t)))

In [ ]:
len(ik.q())

19